In [133]:
from snownlp import SnowNLP
import jieba
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
import re

In [138]:
from core.utils.text_preprocessing import TextPreprocessing

In [139]:
textProcessor = TextPreprocessing()

获取train_data中
加载已有数据
获取train_tokenizer中
加载预训练Tokenizer
获取embedding_matrix中
加载预训练Embedding Matrix


In [152]:
speech = open("./Data/speech.txt","r",encoding="utf-8")
def extract_dictionary(file):
    print(file)
    if file:
        dictionary_list = []
        assert file != None
        txt = file.read()
        assert txt != None
        dictionary_txt = re.findall("{'article'.*?'}", txt, re.S)
        for sub_txt in dictionary_txt:
            tmp_list = re.findall(r"\'(?P<key>.*?)\':\s['|[]?(?P<value>.*)['|]]?", sub_txt)
            tmp_dict = dict(tmp_list)
            tmp_dict["date"] = re.sub("'", "", tmp_dict["date"])
            dictionary_list.append(tmp_dict)
        return dictionary_list

In [153]:
results = extract_dictionary(speech)

<_io.TextIOWrapper name='./Data/speech.txt' mode='r' encoding='utf-8'>


In [155]:
results[0]

{'article': '新华社北京8月30日电国家主席习近平8月30日同厄瓜多尔总统拉索通电话。习近平指出，新冠肺炎疫情发生以来，中厄双方相互支持、同舟共济，体现了两国人民的深情厚谊。中方赞赏厄方高度重视发展对华关系。面对世界百年变局和世纪疫情交织的复杂局面，中厄作为全面战略伙伴，要从战略高度和长远角度看待和发展两国关系，继续相互支持、协调合作，让中厄传统友好历久弥新，使中厄关系成为发展中国家合作标杆。中方愿继续为厄方抗击疫情提供支持和帮助，同厄方开展形式多样的抗疫、疫苗合作。相信在双方共同努力下，中厄关系将得到更大发展。习近平强调，厄瓜多尔是中方共建“一带一路”的重要合作伙伴，双方在基础设施、能源矿产、金融等传统领域合作成果丰硕。中国经济进入新发展阶段，将更加开放、更具活力，这将为厄瓜多尔等国带来新的机遇。中方愿扩大自厄方进口规模，提高双方贸易和投资自由化便利化水平，培育健康、数字、绿色丝绸之路等新增长点，打造更多务实合作成果，更好造福两国人民。中方愿同厄方加强多边事务沟通协调，维护国际公平正义和发展中国家正当权益，推动构建人类命运共同体。拉索表示，我谨代表厄瓜多尔政府和人民再次热烈祝贺中国共产党成立100周年，祝贺中国共产党领导中国人民在包括摆脱贫困和抗击新冠肺炎疫情等问题上取得伟大成就。中方提供的疫苗为厄方抗击疫情提供了至关重要的支持，厄瓜多尔人民对此心怀感激，不会忘记。厄方反对将疫情政治化、污名化，希望同中方继续深化疫苗合作。厄方把中国视为最重要的全面战略伙伴，欢迎更多中方企业赴厄投资合作。',
 'date': '2021-08-30',
 'title': '习近平同厄瓜多尔总统拉索通电话',
 'url': 'http://jhsjk.people.cn/article/32212337'}

In [19]:
sentence_list

['2016“中美旅游年”2月29日在京开幕',
 '国家主席习近平向开幕式致贺词',
 '习近平指出，共同举办2016“中美旅游年”是我去年9月对美国进行国事访问期间双方达成的一项重要成果',
 '值此2016“中美旅游年”开幕之际，我谨代表中国政府和人民，并以我个人的名义，对旅游年的开幕表示热烈的祝贺，对远道而来的美国朋友表示热烈的欢迎',
 '习近平指出，中美建交37年来，两国关系取得历史性发展',
 '近年来，双方共同努力推进中美新型大国关系建设，两国在诸多重要领域、重大国际和地区问题上开展了富有成效的协调与合作，不仅造福两国和两国人民，而且有力促进了世界和平、稳定、繁荣',
 '习近平表示，中美都有灿烂的文化、优美的风光，两国人民都有加深了解、增进友谊的强烈愿望',
 '希望双方以举办旅游年为契机，扩大人员往来，加强文化交流，为中美关系发展培育更为厚实的民意和社会基础',
 '欢迎更多美国朋友来中国旅游',
 '祝2016“中美旅游年”圆满成功']

In [13]:
article_1

'2016“中美旅游年”2月29日在京开幕。国家主席习近平向开幕式致贺词。习近平指出，共同举办2016“中美旅游年”是我去年9月对美国进行国事访问期间双方达成的一项重要成果。值此2016“中美旅游年”开幕之际，我谨代表中国政府和人民，并以我个人的名义，对旅游年的开幕表示热烈的祝贺，对远道而来的美国朋友表示热烈的欢迎。习近平指出，中美建交37年来，两国关系取得历史性发展。近年来，双方共同努力推进中美新型大国关系建设，两国在诸多重要领域、重大国际和地区问题上开展了富有成效的协调与合作，不仅造福两国和两国人民，而且有力促进了世界和平、稳定、繁荣。习近平表示，中美都有灿烂的文化、优美的风光，两国人民都有加深了解、增进友谊的强烈愿望。希望双方以举办旅游年为契机，扩大人员往来，加强文化交流，为中美关系发展培育更为厚实的民意和社会基础。欢迎更多美国朋友来中国旅游。祝2016“中美旅游年”圆满成功。(新华社北京3月1日电)'

In [25]:
#Use Jiaba cut
result_cutted_sentence_in_string= []
for string in sentence_list:
    result_cutted_sentence_in_string.append(' '.join(list(jieba.cut(string))))
    


In [26]:
result_cutted_sentence_in_string

['2016 “ 中 美 旅游 年 ” 2 月 29 日 在京开幕',
 '国家 主席 习近平 向 开幕式 致 贺词',
 '习近平 指出 ， 共同 举办 2016 “ 中 美 旅游 年 ” 是 我 去年 9 月 对 美国 进行 国事访问 期间 双方 达成 的 一项 重要 成果',
 '值此 2016 “ 中 美 旅游 年 ” 开幕 之际 ， 我谨 代表 中国政府 和 人民 ， 并 以 我 个人 的 名义 ， 对 旅游 年 的 开幕 表示 热烈 的 祝贺 ， 对 远道而来 的 美国 朋友 表示 热烈 的 欢迎',
 '习近平 指出 ， 中 美 建交 37 年来 ， 两国关系 取得 历史性 发展',
 '近年来 ， 双方 共同努力 推进 中 美 新型 大国 关系 建设 ， 两国 在 诸多 重要 领域 、 重大 国际 和 地区 问题 上 开展 了 富有成效 的 协调 与 合作 ， 不仅 造福 两国 和 两国人民 ， 而且 有力 促进 了 世界 和平 、 稳定 、 繁荣',
 '习近平 表示 ， 中美 都 有 灿烂 的 文化 、 优美 的 风光 ， 两国人民 都 有 加深了解 、 增进友谊 的 强烈 愿望',
 '希望 双方 以 举办 旅游 年 为 契机 ， 扩大 人员 往来 ， 加强 文化交流 ， 为 中美关系 发展 培育 更为 厚实 的 民意 和 社会 基础',
 '欢迎 更 多 美国 朋友 来 中国 旅游',
 '祝 2016 “ 中 美 旅游 年 ” 圆满成功']

In [27]:
embedding_matrix = textProcessor.embedding_matrix

In [28]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.04141271,  0.53461641, -1.0505358 , ...,  0.06256261,
         0.3142924 ,  2.01107216],
       [-1.34495234,  0.692056  , -1.25638592, ...,  1.50753272,
        -0.26766613,  1.0851773 ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [33]:
# Use SnowNLP to extract key words.
s = SnowNLP(article_1)

In [36]:
tmp = s.summary(5)

In [40]:
result_sentence_list_2 = []
for sentence in tmp:
    result_sentence_list_2.append(' '.join(list(jieba.cut(sentence))))

In [41]:
result_sentence_list_2

['2016 “ 中 美 旅游 年 ” 2 月 29 日 在京开幕',
 '值此 2016 “ 中 美 旅游 年 ” 开幕 之际',
 '共同 举办 2016 “ 中 美 旅游 年 ” 是 我 去年 9 月 对 美国 进行 国事访问 期间 双方 达成 的 一项 重要 成果',
 '祝 2016 “ 中 美 旅游 年 ” 圆满成功',
 '不仅 造福 两国 和 两国人民']

In [108]:
from typing import List

In [116]:
# textProcessor.embedding["中国"]

In [121]:
def smoothing_method(sentence_list: List[List[str]], embedding_size=200, a: float = 1e-3):
    sentence_set = []
    for sentence in sentence_list:  # sentence 是List[str]
        vs = np.zeros(
            embedding_size)  # add all word2vec values into one vector for the sentence
        sentence_length = len(sentence)
        for word in sentence:
            a_value = a / (a + 0.0001)  # smooth inverse frequency, SIF
            try:
                word_embedding = textProcessor.embedding[word]
#                 print(word_embedding)
            except:
                print("出错了")
                word_embedding = np.zeros(200)
            vs = np.add(vs, np.multiply(a_value,word_embedding))  # vs += sif * word_vector
        vs = np.divide(vs, sentence_length)  # weighted average
        sentence_set.append(vs)  # add to our existing re-calculated set of sentences
    
    
    # calculate PCA of this sentence set
    pca = PCA()
    pca.fit(np.array(sentence_set))
    u = pca.components_[0]  # the PCA vector
    u = np.multiply(u, np.transpose(u))  # u x uT

    # pad the vector?  (occurs if we have less sentences than embeddings_size)
    if len(u) < embedding_size:
        for i in range(embedding_size - len(u)):
            u = np.append(u, 0)  # add needed extension for multiplication below

    # resulting sentence vectors, vs = vs -u x uT x vs
    sentence_vecs = []
    for vs in sentence_set:
        sub = np.multiply(u, vs)
        sentence_vecs.append(np.subtract(vs, sub))

    return sentence_set,sentence_vecs

In [122]:
sentence_list = re.findall("(.*?)。",dataset["articles"][0])

In [123]:
sentence_list = [list(jieba.cut(sentence)) for sentence in sentence_list]

In [124]:
result_set,result_vecs = smoothing_method(sentence_list)

In [128]:
tmp = np.random.rand(1,200)

In [131]:
tmp2 = np.multiply(tmp,2)

In [132]:
tmp2

array([[1.10215674, 0.58870234, 1.93022113, 0.02695671, 0.1897214 ,
        1.20312199, 1.33751979, 0.90356681, 1.32721979, 0.87632862,
        1.10389802, 0.04345715, 0.25085491, 1.15381543, 1.75280503,
        0.2425139 , 1.23929737, 0.03815315, 0.19768774, 0.06299306,
        0.44798333, 0.24900866, 1.31624891, 1.85642033, 1.41833508,
        0.00240756, 0.07121435, 0.41387025, 0.20759318, 1.35203002,
        1.24788412, 0.48205375, 0.73628959, 1.8694331 , 0.95975569,
        1.49139307, 0.00393641, 1.70571525, 0.57624458, 1.42027286,
        0.45487374, 1.2799144 , 0.70637435, 0.25022228, 1.89248063,
        1.40598964, 1.40236971, 0.06888766, 0.2004919 , 0.57732347,
        1.35698993, 1.01505682, 1.82487082, 0.76975646, 0.70763003,
        1.85111693, 0.75640849, 1.84858159, 1.41567508, 0.02115123,
        1.21135467, 0.21978676, 0.46262773, 0.50462213, 1.67438286,
        1.59746481, 1.79853788, 1.697656  , 1.12566811, 1.32884299,
        1.13185954, 1.90157506, 1.22248099, 1.26